<a href="https://colab.research.google.com/github/ebartile/binance-multisignature-wallet/blob/main/vision_rl_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
# ----------------------------
# 0. Install dependencies (run once)
# ----------------------------
!pip install torch torchvision opencv-python numpy Pillow PyQt5 gym stable-baselines3[extra] pycocotools ultralytics supervision pandas matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 12.1 MB/s eta 0:00:00


In [3]:
# ----------------------------
# 1. Imports and setup
# ----------------------------
import os
import sys
import cv2
import json
import shutil
import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO
from PyQt5.QtWidgets import QApplication
from IPython.display import display, Image

# Add project root to sys.path if needed
sys.path.append(os.getcwd())

# Paths
DATA_DIR = 'data'
FRAMES_DIR = os.path.join(DATA_DIR, 'frames')
ANNOTATIONS_FILE = os.path.join(DATA_DIR, 'annotations', 'labels.json')
YOLO_DATA_DIR = os.path.join(DATA_DIR, 'yolo_data')
RAW_VIDEOS = [os.path.join(DATA_DIR, f'raw_video{i}.mp4') for i in range(1, 4)]

os.makedirs(FRAMES_DIR, exist_ok=True)
os.makedirs(os.path.dirname(ANNOTATIONS_FILE), exist_ok=True)

In [5]:
# ----------------------------
# 2. Frame extraction
# ----------------------------
def extract_frames(video_path, output_dir, frame_rate=1):
    """Extract frames from a video every N frames (frame_rate)."""
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_dir = os.path.join(output_dir, video_name)
    os.makedirs(video_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {video_path}")
        return

    frame_count = 0
    saved_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_rate == 0:
            frame_filename = os.path.join(video_dir, f"{video_name}_frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1
        frame_count += 1

    cap.release()
    print(f"Extracted {saved_count} frames from {video_path} to {video_dir}")
    return video_dir

# Extract frames from all videos
for video_path in RAW_VIDEOS:
    extract_frames(video_path, FRAMES_DIR, frame_rate=5)  # Adjust frame_rate as needed


Extracted 1387 frames from data/raw_video1.mp4 to data/frames/raw_video1
Extracted 945 frames from data/raw_video2.mp4 to data/frames/raw_video2
Extracted 1317 frames from data/raw_video3.mp4 to data/frames/raw_video3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ----------------------------
# 3. Launch annotation GUI
# ----------------------------
# Note: GUI will open in a separate window; close it after labeling
# This step can only be done on a computer with monitor and not colab